In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# Load data
mask_pos = np.loadtxt('results/limu_model/One_out/pretrain/epoch_2_lr_0.0001_d_hidden_72_d_ff_144_n_heads_4_n_layer_4_embNorm_False/mask_pos.txt')
origin_seq = np.loadtxt('results/limu_model/One_out/pretrain/epoch_2_lr_0.0001_d_hidden_72_d_ff_144_n_heads_4_n_layer_4_embNorm_False/origin_seq.txt')
y_pred = np.loadtxt('results/limu_model/One_out/pretrain/epoch_2_lr_0.0001_d_hidden_72_d_ff_144_n_heads_4_n_layer_4_embNorm_False/pred.txt')
t_true = np.loadtxt('results/limu_model/One_out/pretrain/epoch_2_lr_0.0001_d_hidden_72_d_ff_144_n_heads_4_n_layer_4_embNorm_False/true.txt')

# Split into x and y coordinates
origin_seq_x = origin_seq[:, 0]
origin_seq_y = origin_seq[:, 1]

# Define the number of points per subplot
points_per_subplot = 150
num_subplots = len(origin_seq) // points_per_subplot

# Create a function to plot a specific subplot
def plot_subplot(i):
    start = i * points_per_subplot
    end = start + points_per_subplot

    # Create a figure with 2 subplots
    fig, axs = plt.subplots(2, figsize=(10, 5))

    # Plot x-coordinates
    axs[0].plot(np.arange(start, end), origin_seq_x[start:end], 'r-', label='X-coordinates')
    axs[0].set_title('X-coordinates ({}-{})'.format(start, end))
    axs[0].legend()

    # Plot y-coordinates
    axs[1].plot(np.arange(start, end), origin_seq_y[start:end], 'r-', label='Y-coordinates')
    axs[1].set_title('Y-coordinates ({}-{})'.format(start, end))
    axs[1].legend()

    # Display the plots
    plt.tight_layout()
    plt.show()

# Create a slider to select the subplot
slider = widgets.IntSlider(min=0, max=num_subplots-1, step=1, description='Subplot:')

# Create an interactive plot viewer
widgets.interactive(plot_subplot, i=slider)

interactive(children=(IntSlider(value=0, description='Subplot:', max=45), Output()), _dom_classes=('widget-int…